# 数值计算基础

### ***原则1：矩阵相乘，小维度优先***

矩阵 $A_{n\times p}$，向量 $x_{p\times 1}$，计算 $A^{T}Ax$。

In [1]:
import numpy as np
np.set_printoptions(linewidth=100)

np.random.seed(123)
n = 2000
p = 1000
A = np.random.normal(size=(n, p))
x = np.random.normal(size=p)

方法1：先计算 $A^{T}A$，再与 $x$ 相乘：

In [2]:
%timeit A.transpose().dot(A).dot(x)

57.4 ms ± 2.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


方法2：先计算 $Ax$，再左乘 $A^{T}$：

In [ ]:
%timeit A.transpose().dot(A.dot(x))

经验法则：对于更一般的矩阵乘法 $A_{m\times n}B_{n\times p}C_{p\times r}$，如果 $n\approx p$ 且 $m>r$，则优先计算 $BC$，反之优先计算 $AB$。

In [ ]:
np.random.seed(123)
m = 1000
n = 500
p = 200
r = 100
A = np.random.normal(size=(m, n))
B = np.random.normal(size=(n, p))
C = np.random.normal(size=(p, r))

In [ ]:
%timeit A.dot(B).dot(C)

In [ ]:
%timeit A.dot(B.dot(C))

* 先计算向量有关的运算，再计算矩阵运算（维度小的运算会对数据降维，减少时间复杂度）
* Thm：对于一个n\*p, p\*q的矩阵乘法，时间复杂度为O(nmp)

### 原则2：尽量避免显式矩阵求逆

矩阵 $A_{n\times n}$，向量 $b_{n\times 1}$，计算 $A^{-1}b$。
- 只要不是单独求矩阵的逆，都应该整体作为线性方程组进行求解（其实单独的逆也可以这样求解）

In [ ]:
np.random.seed(123)
n = 1000
A = np.random.normal(size=(n, n))
b = np.random.normal(size=n)

方法1：先计算 $A^{-1}$，再与 $b$ 相乘：

In [ ]:
%timeit np.linalg.inv(A).dot(b)

方法2：解线性方程组 $Ax=b$：

In [ ]:
%timeit np.linalg.solve(A, b)

线性方程组右边也可以是矩阵，即 $A_{n\times n}$，$B_{n\times p}$，计算 $A^{-1}B$。

In [ ]:
np.random.seed(123)
n = 1000
p = 100
A = np.random.normal(size=(n, n))
B = np.random.normal(size=(n, p))

In [ ]:
%timeit np.linalg.inv(A).dot(B)

In [ ]:
%timeit np.linalg.solve(A, B)

### 原则3：利用矩阵的特殊结构

矩阵 $A_{n\times n}$，对角矩阵 $W_{n\times n}$，计算 $WA$ 和 $AW$。
- *对于diag矩阵，甚至存储上都不应该显式以矩阵的形式存储（而是仅存储对角元）*

In [4]:
np.random.seed(123)
n = 1000
A = np.random.normal(size=(n, n))
w = np.random.normal(size=n)
W = np.diag(w)

In [5]:
%timeit W.dot(A)

57.9 ms ± 5.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%timeit A.dot(W)

94.4 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


令 $w_{n\times 1}$ 表示 $W$ 的对角元素，$WA$ 相当于将 $A$ 的每一列乘以 $w$，$AW$ 相当于将 $A$ 的每一行乘以 $w^{T}$。此时可利用 Numpy 的广播机制进行运算。

In [7]:
%timeit A * w.reshape(n, 1)

752 µs ± 59.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
%timeit A * w

854 µs ± 5.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### 练习：回归分析

计算 $\hat{y}=X(X^{T}X)^{-1}X^{T}y$

In [9]:
np.random.seed(123)
n = 2000
p = 500
X = np.random.normal(size=(n, p))
y = np.random.normal(size=n)

### 矩阵运算复杂度

假设$A,B : n\times n, b :n\times1$，则下列运算复杂度：

$AB:O(n^3), Ab:O(n^2)$

$A^{-1}:O(n^3), A^{-1}b:O(n^3)$ （但从实证方面看后者效率更高）

$|A|, eign(A) :O(n^3) $
     - 特别的：上/下三角行列式$O(n)$ （行列式为对角线的乘积）

$||A||_p^2 : O(n^2)$

$ A+b1^T : O(n^2)$ 相当于将$b$广播

## 原则四：尽可能将循环合并为矩阵运算

- 虽然理论复杂度相似，但是显式的循环操作相对而言比矩阵的统一操作耗时更长
- 拓广到分布式计算，当每次通信的粒度越大，通信时间成本越低；反之则越高。